In [1]:
import numpy as np
import pandas as pd
import ast
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.metrics.pairwise import cosine_similarity

In [2]:
anime = pd.read_csv('/Users/ericpark/Desktop/animes.csv')
anime = anime.drop_duplicates(subset=['uid'])

In [3]:
anime = anime.drop(columns = ['aired','episodes','members','score'])

In [4]:
anime

,uid,title,synopsis,genre,popularity,ranked,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",141,25.0,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...",28,24.0,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...",98,23.0,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...",4,1.0,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']",502,22.0,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...
...,...,...,...,...,...,...,...,...
19002,10075,Naruto x UT,All-new animation offered throughout UNIQLO cl...,"['Action', 'Comedy', 'Super Power', 'Martial A...",2382,1728.0,https://cdn.myanimelist.net/images/anime/3/304...,https://myanimelist.net/anime/10075/Naruto_x_UT
19003,35828,Miira no Kaikata,High school student Sora Kashiwagi is accustom...,"['Slice of Life', 'Comedy', 'Supernatural']",1648,1727.0,https://cdn.myanimelist.net/images/anime/1486/...,https://myanimelist.net/anime/35828/Miira_no_K...
19004,10378,Shinryaku!? Ika Musume,"After regaining her squid-like abilities, Ika ...","['Slice of Life', 'Comedy', 'Shounen']",1547,1548.0,https://cdn.myanimelist.net/images/anime/6/301...,https://myanimelist.net/anime/10378/Shinryaku_...
19005,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],2154,1544.0,https://cdn.myanimelist.net/images/anime/12/79...,https://myanimelist.net/anime/33082/Kingsglaiv...


In [14]:
reviews = pd.read_csv('/Users/ericpark/Desktop/reviews.csv')

In [15]:
reviews = reviews.drop_duplicates(subset=['uid'])
reviews = reviews.drop(columns=['text','scores','link'])

In [16]:
reviews = reviews.groupby(['profile','uid','anime_uid','score']).count().reset_index()

In [17]:
reviews

,profile,uid,anime_uid,score
0,-----noname-----,210287,18441,2
1,-----noname-----,221727,2025,4
2,---SnowFlake---,208633,1535,6
3,---was-----,79296,10110,8
4,--EYEPATCH--,274580,35839,10
...,...,...,...,...
130514,zzz-anime,319298,36882,2
130515,zzzb,272281,28761,8
130516,zzzjynne,322680,6675,7
130517,zzzjynne,322740,18679,8


I wanted to get recommendations given a certain anime by seeing which users liked that anime and then seeing what other animes they liked. I then see which anime are also most liked among users who liked a certain anime. One problem that I foresee about this method is that it favors more mainstream anime and not necessarily the most "similar" anime. 

In [8]:
def getUsersAbove8(anime_uid):#given animeid, return a list of users who scored the anime an 8 or above
    anime = reviews[reviews['anime_uid']==anime_uid]
    return anime[anime['score']>=8]['profile'].tolist()

I made 8 as the minimum score for someone to be considered to "like" an anime

In [9]:
def getAnimeListAbove8(username, uid):#given user, return a list of anime the user scored an 8 or above
    user = reviews[reviews['profile']==username]
    animelist=user[user['score']>=8]['anime_uid'].tolist()
    animelist.remove(uid)
    return animelist

In [10]:
def getUid(anime_title):
    return anime[anime['title']==anime_title]['uid'].iloc[0]

At this point, I wanted to narrow down the anime by similar genre. I ran into a problem because I wanted to use cosine similarity or a similar concept, but on MyAnimeList(the website I scraped the data from), every anime has more than one genre and I wasn't sure how to deal with that. I am looking into this for future drafts.

In [11]:
def getGenre(anime_uid):
    return ast.literal_eval(anime[anime['uid']==anime_uid]['genre'].iloc[0])

In [12]:
def getAnimeTitle(anime_uid):
    return anime[anime['uid']==anime_uid]['title'].iloc[0]

In [13]:
def getRecommendations(anime_title):   
    if (len(anime[anime['title']==anime_title])==0):#if it is not found
        return "Anime not found. Make sure it is spelled exactly how it appears"
    else:
        #get uid
        anime_uid = getUid(anime_title) #anime uid
        #original anime genre
        original_genre = getGenre(anime_uid) #list of genres for anime
        #find users who scored anime an 8 or above
        userlist = getUsersAbove8(anime_uid) #list of users who scored anime an 8 or above
        #within those users, find all the anime they rated an 8 or above
        animelist=[]
        for i in userlist:
            animelist.append(getAnimeListAbove8(i,anime_uid))
        #filter down to anime with similar genre
        final_list = []
        for user in animelist:
            for anime_user in user:
                if(len(sorted(set(original_genre).intersection(getGenre(anime_user))))>1):#if they share more than 1 genre
                    final_list.append(anime_user)
        anime_df = pd.DataFrame(final_list,columns=['Anime_UID'])
        anime_df.insert(1,"Count",1)
        anime_df = anime_df.groupby("Anime_UID").count().sort_values(by=['Count'],ascending=False)#df of top anime that match
        anime_top10 = anime_df.index.values.tolist()[:10]#get top 10 by count
        for i in anime_top10:
            print(getAnimeTitle(i))

In [15]:
getRecommendations("Fullmetal Alchemist: Brotherhood")

Cowboy Bebop
Clannad: After Story
Fullmetal Alchemist
Mahou Shoujo Madoka★Magica
Tengen Toppa Gurren Lagann
Code Geass: Hangyaku no Lelouch
Baccano!
Code Geass: Hangyaku no Lelouch R2
Sen to Chihiro no Kamikakushi
Boku no Hero Academia
